In [3]:
import pandas as pd

movies_df = pd.read_csv('/content/movies (2).csv')
ratings_df = pd.read_csv('ratings.csv')

display(movies_df.head())
display(ratings_df.head())


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [5]:

merged_df = pd.merge(ratings_df, movies_df, on='movieId')

display(merged_df.head())


,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,1,24,1.5,1217895807,Powder (1995),Drama|Sci-Fi
2,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,4.0,1217896556,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,4.0,1217896523,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:

genre_input = 'Comedy'
genre_movies = merged_df[merged_df['genres'].str.contains(genre_input)]
display(genre_movies.head())


,userId,movieId,rating,timestamp,title,genres
10,1,223,4.0,1217897795,Clerks (1994),Comedy
11,1,256,0.5,1217895764,Junior (1994),Comedy|Sci-Fi
15,1,296,4.0,1217896125,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
18,1,356,3.0,1217896231,Forrest Gump (1994),Comedy|Drama|Romance|War
20,1,380,3.0,1217896030,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller


In [41]:

movie_review_counts = genre_movies.groupby('movieId').size()
t = 100
popular_movies = genre_movies[genre_movies['movieId'].isin(movie_review_counts[movie_review_counts >= t].index)]

display(popular_movies.head())


,userId,movieId,rating,timestamp,title,genres
10,1,223,4.0,1217897795,Clerks (1994),Comedy
15,1,296,4.0,1217896125,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
18,1,356,3.0,1217896231,Forrest Gump (1994),Comedy|Drama|Romance|War
20,1,380,3.0,1217896030,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller
28,1,597,3.0,1217897176,Pretty Woman (1990),Comedy|Romance


In [11]:

avg_ratings = popular_movies.groupby('movieId')['rating'].mean()

display(avg_ratings.head())


,rating
movieId,
1,3.907328
21,3.490741
141,3.551887
153,2.964968
223,3.980198


In [13]:

N = 5
top_movies = avg_ratings.sort_values(ascending=False).head(N)

top_movies_with_details = pd.merge(top_movies, movies_df, on='movieId')

display(top_movies_with_details[['title', 'rating']])


,title,rating
0,Monty Python and the Holy Grail (1975),4.301948
1,Fargo (1996),4.271144
2,"Princess Bride, The (1987)",4.163743
3,Pulp Fiction (1994),4.160000
4,Forrest Gump (1994),4.138264


In [20]:

movies_df['genres'] = movies_df['genres'].astype(str).str.split('|')

genres = movies_df['genres'].explode().unique()
for genre in genres:
    movies_df[genre] = movies_df['genres'].apply(lambda x: 1 if genre in x else 0)

display(movies_df.head())

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Sci-Fi,IMAX,War,Musical,Documentary,Western,Film-Noir,(no genres listed),nan,['nan']
0,1,Toy Story (1995),[['nan']],1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,1,1
1,2,Jumanji (1995),[['nan']],1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,3,Grumpier Old Men (1995),[['nan']],0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,1
3,4,Waiting to Exhale (1995),[['nan']],0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1,1
4,5,Father of the Bride Part II (1995),[['nan']],0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [42]:
from sklearn.metrics.pairwise import cosine_similarity

input_movie_title = 'Toy Story'

input_movie = movies_df[movies_df['title'] == input_movie_title]

if input_movie.empty:
    print(f"Movie '{input_movie_title}' not found.")
else:
    input_movie_vector = input_movie[genres].values



    movie_vectors = movies_df[genres].values
    similarities = cosine_similarity(input_movie_vector, movie_vectors)


    movies_df['similarity'] = similarities.flatten()

    print(movies_df[['title', 'similarity']].head())


Movie 'Toy Story' not found.


In [43]:

movie_review_counts = ratings_df.groupby('movieId').size()

t = 100

filtered_movies = movies_df[movies_df['movieId'].isin(movie_review_counts[movie_review_counts >= t].index)]

display(filtered_movies[['title']].head())

,title
0,Toy Story (1995)
5,Heat (1995)
9,GoldenEye (1995)
16,Sense and Sensibility (1995)
20,Get Shorty (1995)


In [28]:

N = 5

top_similar_movies = filtered_movies.sort_values(by='title', ascending=False).head(N)

display(top_similar_movies[['title']])


,title
2999,X-Men (2000)
740,"Wizard of Oz, The (1939)"
872,Willy Wonka & the Chocolate Factory (1971)
2388,Who Framed Roger Rabbit? (1988)
299,While You Were Sleeping (1995)


In [32]:

user_movie_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating')

display(user_movie_matrix.head())


movieId,1,2,3,4,5,6,7,8,9,10,...,144482,144656,144976,146344,146656,146684,146878,148238,148626,149532
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

user_movie_matrix_filled = user_movie_matrix.fillna(0)

user_similarity_matrix = cosine_similarity(user_movie_matrix_filled)

user_similarity_df = pd.DataFrame(user_similarity_matrix, index=user_movie_matrix_filled.index, columns=user_movie_matrix_filled.index)

display(user_similarity_df.head())


userId,1,2,3,4,5,6,7,8,9,10,...,659,660,661,662,663,664,665,666,667,668
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.101113,0.210044,0.128766,0.057896,0.077130,0.358090,0.097434,0.239189,0.026663,...,0.291162,0.144741,0.106583,0.091049,0.236805,0.154519,0.245071,0.238660,0.278217,0.153479
2,0.101113,1.000000,0.115559,0.034610,0.032705,0.028305,0.062914,0.471918,0.194232,0.000000,...,0.068325,0.000000,0.477330,0.146887,0.163553,0.061737,0.050948,0.051423,0.035907,0.064816
3,0.210044,0.115559,1.000000,0.058208,0.044426,0.012816,0.084522,0.066620,0.459703,0.068454,...,0.152078,0.301021,0.081626,0.098949,0.310234,0.079452,0.092821,0.080940,0.158943,0.109648
4,0.128766,0.034610,0.058208,1.000000,0.019298,0.005781,0.059089,0.024420,0.050572,0.000000,...,0.055860,0.024329,0.040467,0.108881,0.076241,0.014011,0.042643,0.174275,0.061677,0.157794
5,0.057896,0.032705,0.044426,0.019298,1.000000,0.053378,0.080822,0.041536,0.023168,0.011915,...,0.058450,0.007315,0.024708,0.038163,0.053085,0.048993,0.055431,0.026053,0.086667,0.068275


In [36]:

target_user_id = 1
K = 100

similarity_scores = user_similarity_df[target_user_id].drop(target_user_id)

top_K_similar_users = similarity_scores.sort_values(ascending=False).head(K)

display(top_K_similar_users)


,1
userId,
348,0.413838
506,0.411054
493,0.406140
158,0.404324
633,0.402729
...,...
632,0.277360
604,0.276816
290,0.276332


In [44]:

similar_users_ratings = ratings_df[ratings_df['userId'].isin(top_K_similar_users.index)]

similar_users_ratings_matrix = similar_users_ratings.pivot(index='userId', columns='movieId', values='rating')

predicted_ratings = {}

for movie_id in similar_users_ratings_matrix.columns:

    movie_ratings = similar_users_ratings_matrix[movie_id]


    rated_users = movie_ratings.dropna()

    if len(rated_users) > 0:

        weighted_sum = sum(rated_users * top_K_similar_users.loc[rated_users.index])
        similarity_sum = sum(top_K_similar_users.loc[rated_users.index])


        predicted_rating = weighted_sum / similarity_sum if similarity_sum != 0 else 0
        predicted_ratings[movie_id] = predicted_rating


predicted_ratings_df = pd.DataFrame(predicted_ratings.items(), columns=['movieId', 'predicted_rating'])
display(predicted_ratings_df.head())

,movieId,predicted_rating
0,1,3.964063
1,2,3.283486
2,3,3.593268
3,5,2.915074
4,6,4.185698


In [40]:

N = 5

top_N_recommended_movies = predicted_ratings_df.sort_values(by='predicted_rating', ascending=False).head(N)

top_N_recommended_movies_with_titles = pd.merge(top_N_recommended_movies, movies_df[['movieId', 'title']], on='movieId')

display(top_N_recommended_movies_with_titles[['title', 'predicted_rating']])


,title,predicted_rating
0,"Grand Budapest Hotel, The (2014)",5.0
1,Billy Elliot (2000),5.0
2,Show Me Love (Fucking Åmål) (1998),5.0
3,"Nasty Girl, The (schreckliche Mädchen, Das) (1...",5.0
4,Catch-22 (1970),5.0
